In [29]:
from terra_sdk.client.lcd import LCDClient
from datetime import datetime
import pandas as pd
import numpy as np
import requests
import plotly.graph_objects as go
import plotly.express as px
from tools import get_validator_df, create_stablecoin_df, total_token_supply
import warnings
warnings.filterwarnings('ignore')
terra = LCDClient(chain_id="columbus-5", url="https://lcd.terra.dev")

## This Page gives various tools to better visualize the health of the Terra Ecosystem

In [6]:
# Sets the numeric option to remove scientific notation
pd.set_option('display.float_format', lambda x: '%.6f' % x)

### Wallet Account Growth 

In [58]:
# shows the wallet growth from the genesis block on 
account_growth = requests.get('https://fcd.terra.dev/v1/dashboard/account_growth').json()


# One day is equivalent to 86400000 ms
day_divider = 86400000

account_df = pd.DataFrame(account_growth['cumulative'])
account_df['datetime'] = (account_df['datetime']/day_divider).values.astype(dtype='datetime64[D]') # for day format

In [61]:
# Charts a graph for wallet growth over time 

fig = px.line(account_df, x = 'datetime', y = 'totalAccountCount', title='Wallet Account Growth')

fig.update_xaxes(
    rangeslider_visible=False,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()

### Validator Diversity 

In [8]:
v_df = get_validator_df()
v_df

,operatorAddress,tokens,delegatorShares,upTime,status,accountAddress,votingPower,commissionInfo,goodKarma
description,,,,,,,,,
Orion.Money,terravaloper1259cmu5zyklsdkmgstxhwqpe0utfe5hhy...,21766903.837134,21766903.837134,0.999986,active,terra1259cmu5zyklsdkmgstxhwqpe0utfe5hhyygjdc,0.0671037432,0.0500000000,0.000000
B-Harvest,terravaloper15zcjduavxc5mkp8qcqs9eyhwlqwdlrzy6...,15433802.556738,15433802.556738,0.999984,active,terra15zcjduavxc5mkp8qcqs9eyhwlqwdlrzy6anwpg,0.0475798457,0.0500000000,0.000000
Staking Fund,terravaloper123gn6j23lmexu0qx5qhmgxgunmjcqsx8g...,13630920.312331,13630920.312331,0.999986,active,terra123gn6j23lmexu0qx5qhmgxgunmjcqsx8g5ueq2,0.0420218602,0.1000000000,0.000000
DokiaCapital,terravaloper1v5hrqlv8dqgzvy0pwzqzg0gxy899rm4kd...,10507009.413597,10507009.413597,0.999995,active,terra1v5hrqlv8dqgzvy0pwzqzg0gxy899rm4kdn0jp4,0.0323913620,0.0500000000,0.000000
everstakeone,terravaloper13g7z3qq6f00qww3u4mpcs3xw5jhqwrasw...,8708674.870278,8712117.605403,0.999995,active,terra13g7z3qq6f00qww3u4mpcs3xw5jhqwraswv3q3t,0.0268473941,0.1000000000,3442.735125
...,...,...,...,...,...,...,...,...,...
bc1,terravaloper18p7kdpvpsp3muz93ww7ej72wyrrdja2l7...,307575.099909,307636.624034,0.999988,active,terra18p7kdpvpsp3muz93ww7ej72wyrrdja2l7jjxe9,0.0009482026,1.0000000000,61.524125
✅ CryptoCrew Validators #IBCgang,terravaloper13slfa8cc7zvmjt4wkap2lwmlkp4h3azwl...,303835.328678,303835.328678,0.999824,active,terra13slfa8cc7zvmjt4wkap2lwmlkp4h3azwlyn02r,0.0009366728,0.0500000000,0.000000
FreshLUNA.com Enterprise Validator,terravaloper1audgfvmgt0js54p3s8kj3r40uwej6vy2t...,274801.683318,274801.683318,0.999998,active,terra1audgfvmgt0js54p3s8kj3r40uwej6vy2trk7na,0.0008471689,0.0899000000,0.000000


In [26]:
# Charts the distribution of each validators share in the network 
# to better illustrate how diversified the validators are
import plotly.graph_objects as go
labels = v_df.index
share = (v_df['votingPower'])

fig = go.Figure(data=[go.Pie(labels = labels, values = share, hole = .4)])
fig.update_traces(title="Validator Share of Network")
fig.update_traces(title_position="top left")
fig.update_traces(textinfo= "none")

fig.show()

In [10]:
# The 'goodKarma' column is created by measuring the difference from tokens and delegator shares
# To see which validators have shares delegated to them 
# this DataFrame is organized by the validators with the highest amount of delegated shares
top10delegatedshares = v_df.sort_values(by= 'goodKarma', ascending=False).head(10)
top10delegatedshares

,operatorAddress,tokens,delegatorShares,upTime,status,accountAddress,votingPower,commissionInfo,goodKarma
description,,,,,,,,,
everstakeone,terravaloper13g7z3qq6f00qww3u4mpcs3xw5jhqwrasw...,8708674.870278,8712117.605403,0.999995,active,terra13g7z3qq6f00qww3u4mpcs3xw5jhqwraswv3q3t,0.0268473941,0.1000000000,3442.735125
hashed,terravaloper1p54hc4yy2ajg67j645dn73w3378j6k05v...,7674868.274856,7675635.837215,0.999998,active,terra1p54hc4yy2ajg67j645dn73w3378j6k05v52cnk,0.0236603421,0.1000000000,767.562359
Staked,terravaloper1h6rf7y2ar5vz64q8rchz5443s3tqnswrp...,6005996.810382,6006597.469594,0.999900,active,terra1h6rf7y2ar5vz64q8rchz5443s3tqnswrpxe69f,0.0185157515,0.1000000000,600.659212
DELIGHT,terravaloper1fjuvyccn8hfmn5r7wc2t3kwqy09zzp6ty...,5369998.829177,5370535.882330,0.999549,active,terra1fjuvyccn8hfmn5r7wc2t3kwqy09zzp6tya55yu,0.0165548111,0.1000000000,537.053153
01node,terravaloper1khfcg09plqw84jxy5e7fj6ag4s2r9wqsg...,5046880.240724,5047384.978452,0.999995,active,terra1khfcg09plqw84jxy5e7fj6ag4s2r9wqsg5jt4x,0.0155589141,0.0800000000,504.737728
Terra Bites,terravaloper1t90gxaawul292g2vvqnr3g0p39tw5v6vs...,4167707.421430,4168124.229089,0.999949,active,terra1t90gxaawul292g2vvqnr3g0p39tw5v6vsecu4f,0.0128483478,0.0300000000,416.807659
InfStones,terravaloper18jv83kckd0r8ht9fwsqzyvljcd4h4pj9u...,2951389.074544,2951684.242953,0.999963,active,terra18jv83kckd0r8ht9fwsqzyvljcd4h4pj9uxlk72,0.0090986416,0.0800000000,295.168409
block42,terravaloper16tc3c9u6yj5uuhru32pvs0pahfwraurpy...,1165981.084967,1166214.315902,0.999991,active,terra16tc3c9u6yj5uuhru32pvs0pahfwraurpywwrup,0.0035945769,0.1000000000,233.230935
Staky.io,terravaloper1lm604mtdalpd9z46486acqlshns2urquw...,148741.115981,148889.938600,0.999924,active,terra1lm604mtdalpd9z46486acqlshns2urquwr6xs5,0.0004585503,0.0800000000,148.822619


### Total Token Supply

In [57]:
# Creates the token supply dataframe and changes labels to a more readable format
token_supply = total_token_supply()
token_supply['denom'][3] = "xrowan"
token_supply['denom'][22] = "uscrt"
token_supply['denom'][25] = "xust"
token_supply = token_supply.loc[token_supply['amount'] > 1000]

token_supply

,denom,amount
3,xrowan,120000.000000
22,uscrt,16529.656215
25,xust,5075909.186109
28,uaud,12852.592534
29,ucad,6696.719856
30,uchf,9042.439698
31,ucny,21084.537958
32,udkk,8110.429778
33,ueur,678849.311483
34,ugbp,4453.325243


### Stablecoin Growth by transaction volume

In [14]:
# Create a DataFrame for each Stablecoin
uusd_df = create_stablecoin_df(0)
utwd_df = create_stablecoin_df(1)
uthb_df = create_stablecoin_df(2)
usgd_df = create_stablecoin_df(3)
usek_df = create_stablecoin_df(4)
usdr_df = create_stablecoin_df(5)
uphp_df = create_stablecoin_df(6)
unok_df = create_stablecoin_df(7)
umyr_df = create_stablecoin_df(8)
umnt_df = create_stablecoin_df(9)
uluna_df = create_stablecoin_df(10)
ukrw_df = create_stablecoin_df(11)
ujpy_df = create_stablecoin_df(12)
uinr_df = create_stablecoin_df(13)
uidr_df = create_stablecoin_df(14)
uhkd_df = create_stablecoin_df(15)
ugbp_df = create_stablecoin_df(16)
ueur_df = create_stablecoin_df(17)
udkk_df = create_stablecoin_df(18)
ucny_df = create_stablecoin_df(19)
uchf_df = create_stablecoin_df(20)
ucad_df = create_stablecoin_df(21)
uaud_df = create_stablecoin_df(22)


In [15]:
# Use cell to check to make sure you are using the correct data
uusd_df

,datetime,txVolume,normTxVolume
0,2019-06-05,0.000000,0.000000
1,2019-06-06,0.000000,0.000000
2,2019-06-07,0.000000,0.000000
3,2019-06-08,0.000000,0.000000
4,2019-06-09,0.000000,0.000000
...,...,...,...
975,2022-02-22,61803350834.047150,867684255.789909
976,2022-02-23,62622479694.993599,879184363.899174
977,2022-02-24,63611257884.120789,893066253.078255
978,2022-02-25,64383877852.443359,903913402.515549


In [28]:
# Compare the cumulative volume change priced in Luna on the Terra network
# Click the legend on the right to toggle on/off on the chart
fig = go.Figure()

fig.add_trace(go.Scatter(x=uusd_df['datetime'], y=uusd_df['normTxVolume'],
                         mode='lines',
                         name='UST')) # 0
fig.add_trace(go.Scatter(x=utwd_df['datetime'], y=utwd_df['normTxVolume'],
                         mode='lines',
                         name='TWT')) # 1
fig.add_trace(go.Scatter(x=uthb_df['datetime'], y=uthb_df['normTxVolume'],
                         mode='lines',
                         name='THT')) # 2
fig.add_trace(go.Scatter(x=usgd_df['datetime'], y=usgd_df['normTxVolume'],
                         mode='lines',
                         name='SGT')) # 3
fig.add_trace(go.Scatter(x=usek_df['datetime'], y=usek_df['normTxVolume'],
                         mode='lines',
                         name='SET')) # 4
fig.add_trace(go.Scatter(x=usdr_df['datetime'], y=usdr_df['normTxVolume'],
                         mode='lines',
                         name='SDT')) # 5
fig.add_trace(go.Scatter(x=uphp_df['datetime'], y=uphp_df['normTxVolume'],
                         mode='lines',
                         name='PHT')) # 6
fig.add_trace(go.Scatter(x=unok_df['datetime'], y=unok_df['normTxVolume'],
                         mode='lines',
                         name='NOT')) # 7
fig.add_trace(go.Scatter(x=umyr_df['datetime'], y=umyr_df['normTxVolume'],
                         mode='lines',
                         name='MYT')) # 8
fig.add_trace(go.Scatter(x=umnt_df['datetime'], y=umnt_df['normTxVolume'],
                         mode='lines',
                         name='MNT')) # 9
# fig.add_trace(go.Scatter(x=uluna_df['datetime'], y=uluna_df['normTxVolume'],
#                          mode='lines',
#                          name='LUNA')) # 10
fig.add_trace(go.Scatter(x=ukrw_df['datetime'], y=ukrw_df['normTxVolume'],
                         mode='lines',
                         name='KRT')) # 11
fig.add_trace(go.Scatter(x=ujpy_df['datetime'], y=ujpy_df['normTxVolume'],
                         mode='lines',
                         name='JPT')) # 12
fig.add_trace(go.Scatter(x=uinr_df['datetime'], y=uinr_df['normTxVolume'],
                         mode='lines',
                         name='INT')) # 13
fig.add_trace(go.Scatter(x=uidr_df['datetime'], y=uidr_df['normTxVolume'],
                         mode='lines',
                         name='IDT')) # 14
fig.add_trace(go.Scatter(x=uhkd_df['datetime'], y=uhkd_df['normTxVolume'],
                         mode='lines',
                         name='HKT')) # 15
fig.add_trace(go.Scatter(x=ugbp_df['datetime'], y=ugbp_df['normTxVolume'],
                         mode='lines',
                         name='GBT')) # 16
fig.add_trace(go.Scatter(x=ueur_df['datetime'], y=ueur_df['normTxVolume'],
                         mode='lines',
                         name='EUT')) # 17
fig.add_trace(go.Scatter(x=udkk_df['datetime'], y=udkk_df['normTxVolume'],
                         mode='lines',
                         name='UDT')) # 18
fig.add_trace(go.Scatter(x=ucny_df['datetime'], y=ucny_df['normTxVolume'],
                         mode='lines',
                         name='CNT')) # 19
fig.add_trace(go.Scatter(x=uchf_df['datetime'], y=uchf_df['normTxVolume'],
                         mode='lines',
                         name='CHT')) # 20
fig.add_trace(go.Scatter(x=ucad_df['datetime'], y=ucad_df['normTxVolume'],
                         mode='lines',
                         name='CAT')) # 21
fig.add_trace(go.Scatter(x=uaud_df['datetime'], y=uaud_df['normTxVolume'],
                         mode='lines',
                         name='AUT')) # 22
fig.update_layout(title = "Stablecoin Growth (priced in LUNA for normalization)")
fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig.show()